In [1]:
import pandas as pd
import zipfile

In [2]:
def load_data(file_path: str) -> pd.DataFrame:
        return pd.read_csv(file_path, sep = '\t')

In [10]:
df = load_data('data/eu_life_expectancy_raw.tsv')

In [4]:
df

,"unit,sex,age,geo\time",2021,2020,2019,2018,2017,2016,2015,2014,2013,...,1969,1968,1967,1966,1965,1964,1963,1962,1961,1960
0,"YR,F,Y1,AL",:,79.4,80.4,80.2,79.7,79.8,79.2,79.8,79.6,...,:,:,:,:,:,:,:,:,:,:
1,"YR,F,Y1,AM",:,:,79.1,79.2,78.5,78.0,77.9,:,:,...,:,:,:,:,:,:,:,:,:,:
2,"YR,F,Y1,AT",:,82.9,83.5,83.3,83.2,83.4,83.0,83.3,83.0,...,:,:,:,:,:,:,:,:,:,:
3,"YR,F,Y1,AZ",:,:,78.6,78.1,77.7,:,77.5,77.0,76.9,...,:,:,:,:,:,:,:,:,:,:
4,"YR,F,Y1,BE",:,82.3,83.6,83.2,83.2,83.2,82.6,83.1,82.5,...,74.4,74.3,74.7,74.4,74.3,74.6,73.9,74.0,74.5,73.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14443,"YR,T,Y_LT1,SM",:,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,:,:
14444,"YR,T,Y_LT1,TR",:,:,79.1,78.9,78.5,78.1,78.2,78.1,78.2,...,:,:,:,:,:,:,:,:,:,:
14445,"YR,T,Y_LT1,UA",:,:,73.4,73.2,73.3,72.9,72.5,71.8,:,...,:,:,:,:,:,:,:,:,:,:
14446,"YR,T,Y_LT1,UK",:,:,:,81.3,81.3,81.2,81.0,81.4,81.1,...,:,:,:,:,:,:,:,:,:,:


In [11]:
def clean_data(df_expectancy: pd.DataFrame, country: str = 'PT') -> pd.DataFrame:
    """
        Function to clean data
    """

    df_copy = df_expectancy.copy()

    # Splitting the name of the first column by ','
    new_cols = df_copy.columns[0].split(',')

    # Splitting the data of the first column and dropping the column with a bad format
    df_copy[new_cols] = df_copy[df_copy.columns[0]].str.split(',', expand=True)

    df_copy.drop(df_copy.columns[0], axis = 1, inplace = True)

    # Creating a table with unit,sex,age,region,year,value as columns
    df_ = df_copy.melt(id_vars = new_cols, var_name = 'year', value_name = 'value')

    df_.rename(columns={'geo\\time': 'region'}, inplace=True)

    df_['year'] = df_['year'].astype(int)

    # allowing only floats as values to 'value' column
    df_['value'] = (df_['value'].str.extract(r'(\d+\.?\d*)').astype(float))

    df_.dropna(subset=['value'], inplace = True)

    # filtering by the desired country
    # dataframe = df_[df_['region'] == country]

    return df_ #dataframe

In [12]:
df = clean_data(df)

In [13]:
df

,unit,sex,age,region,year,value
3418,YR,F,Y65,AT,2021,21.2
3420,YR,F,Y65,BE,2021,22.2
3421,YR,F,Y65,BG,2021,15.6
3423,YR,F,Y65,CH,2021,23.1
3424,YR,F,Y65,CY,2021,21.2
...,...,...,...,...,...,...
895735,YR,T,Y_LT1,EE,1960,69.4
895749,YR,T,Y_LT1,HU,1960,68.1
895762,YR,T,Y_LT1,NO,1960,73.8
895764,YR,T,Y_LT1,PT,1960,64.0


In [14]:
df['region'].unique()

array(['AT', 'BE', 'BG', 'CH', 'CY', 'CZ', 'DK', 'EE', 'EL', 'ES',
       'EU27_2020', 'FI', 'FR', 'HR', 'HU', 'IS', 'IT', 'LI', 'LT', 'LU',
       'LV', 'MT', 'NL', 'NO', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK', 'DE',
       'DE_TOT', 'AL', 'EA18', 'EA19', 'EFTA', 'IE', 'ME', 'MK', 'RS',
       'AM', 'AZ', 'GE', 'TR', 'UA', 'BY', 'EEA30_2007', 'EEA31',
       'EU27_2007', 'EU28', 'UK', 'XK', 'FX', 'MD', 'SM', 'RU'],
      dtype=object)

In [3]:
def load_data(file_path: str) -> pd.DataFrame:
        with zipfile.ZipFile(file_path, "r") as zip_ref:
            zip_ref.extractall('/nfs/backup/wb_mci_crp/fcalves/assignments/life_expectancy/data')

        return pd.read_json('/nfs/backup/wb_mci_crp/fcalves/assignments/life_expectancy/data/eurostat_life_expect.json')

In [4]:
df2 = load_data('/nfs/backup/wb_mci_crp/fcalves/assignments/life_expectancy/data/eurostat_life_expect.zip')

In [8]:
df2

,unit,sex,age,country,year,life_expectancy,flag,flag_detail
0,YR,F,Y65,AT,2021,21.2,e,estimated
1,YR,F,Y65,BE,2021,22.2,e,estimated
2,YR,F,Y65,BG,2021,15.6,e,estimated
3,YR,F,Y65,CH,2021,23.1,e,estimated
4,YR,F,Y65,CY,2021,21.2,e,estimated
...,...,...,...,...,...,...,...,...
474377,YR,T,Y_LT1,EE,1960,69.4,,None
474378,YR,T,Y_LT1,HU,1960,68.1,,None
474379,YR,T,Y_LT1,NO,1960,73.8,,None
474380,YR,T,Y_LT1,PT,1960,64.0,,None


In [9]:
df2.drop(columns='flag', index = 1)

,unit,sex,age,country,year,life_expectancy,flag_detail
0,YR,F,Y65,AT,2021,21.2,estimated
2,YR,F,Y65,BG,2021,15.6,estimated
3,YR,F,Y65,CH,2021,23.1,estimated
4,YR,F,Y65,CY,2021,21.2,estimated
5,YR,F,Y65,CZ,2021,18.7,estimated
...,...,...,...,...,...,...,...
474377,YR,T,Y_LT1,EE,1960,69.4,None
474378,YR,T,Y_LT1,HU,1960,68.1,None
474379,YR,T,Y_LT1,NO,1960,73.8,None
474380,YR,T,Y_LT1,PT,1960,64.0,None


In [22]:
df2.dtypes

unit                object
sex                 object
age                 object
country             object
year                 int64
life_expectancy    float64
flag                object
flag_detail         object
dtype: object

In [23]:
df.dtypes

unit       object
sex        object
age        object
region     object
year        int64
value     float64
dtype: object